# Data Queries on BART System Stations, Lines, and Travel Times Tables for Graph Creation

In [1]:
import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
cursor = connection.cursor()

#  1. Query the list of all segments between each station and its adjoining stations

In [5]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,line,from_station,to_station,travel_time
0,blue,16th Street Mission,24th Street Mission,120
1,blue,24th Street Mission,Glen Park,180
2,blue,Balboa Park,Daly City,240
3,blue,Bay Fair,San Leandro,240
4,blue,Castro Valley,Bay Fair,240
...,...,...,...,...
103,yellow,Rockridge,MacArthur,240
104,yellow,San Bruno,SFO,240
105,yellow,South San Francisco,San Bruno,240
106,yellow,Walnut Creek,Lafayette,300


# 2. Query the list of all segments between each station and its adjoining stations along with average exit counts. We take the from_station as the reference and calculate the exit counts

In [6]:
rollback_before_flag = True
rollback_after_flag = True

query = """
SELECT DISTINCT
    q.from_station,
    q.to_station,
    q.travel_time,
    be.average_exit_count
FROM stage_bart_exits be
LEFT JOIN stations s ON be.station = s.station
 JOIN (
    SELECT DISTINCT a.line, a.station as from_station, b.station as to_station, t.travel_time
    FROM lines a
    JOIN lines b ON a.line = b.line AND b.sequence = (a.sequence + 1)
    JOIN travel_times t ON (a.station = t.station_1 AND b.station = t.station_2)
        OR (a.station = t.station_2 AND b.station = t.station_1)
) q ON be.station = q.to_station
ORDER BY be.average_exit_count desc;
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,from_station,to_station,travel_time,average_exit_count
0,West Oakland,Embarcadero,420,15097
1,Embarcadero,Montgomery Street,60,12691
2,Montgomery Street,Powell Street,120,10206
3,Powell Street,Civic Center,60,7888
4,Civic Center,16th Street Mission,180,5135
5,16th Street Mission,24th Street Mission,120,4915
6,Balboa Park,Daly City,240,3885
7,Glen Park,Balboa Park,120,3539
8,Coliseum,Fruitvale,240,3464
9,Lake Merritt,Fruitvale,300,3464


In [7]:
rollback_before_flag = True
rollback_after_flag = True

query = """
SELECT *
FROM stage_bart_exits
WHERE station IN ('Coliseum', 'South Hayward', 'Warm Springs', 'Fremont', 'Union City', 'Milpitas', 'San Leandro', 'Fruitvale', 'Hayward', 'MacArthur');
"""
my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id,station_code,station,average_exit_count
0,12,CL,Coliseum,2376
1,21,FM,Fremont,1734
2,22,FV,Fruitvale,3464
3,24,HY,Hayward,1953
4,27,MA,MacArthur,3279
5,29,ML,Milpitas,919
6,42,SH,South Hayward,1223
7,43,SL,San Leandro,2833
8,46,UC,Union City,1583
9,50,WS,Warm Springs,1079


# 3. Query the list of all possible line transfers and the transfer times, create a relationship for each transfer with the transfer time as the weight

In [8]:
rollback_before_flag = True
rollback_after_flag = True

query = """
SELECT a.station, a.line AS from_line, b.line AS to_line, s.transfer_time
FROM lines a
JOIN lines b
  ON a.station = b.station AND a.line <> b.line 
JOIN stations s
  ON a.station = s.station
ORDER BY a.station, a.line, b.line
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,from_line,to_line,transfer_time
0,12th Street,orange,red,282
1,12th Street,orange,yellow,282
2,12th Street,red,orange,282
3,12th Street,red,yellow,282
4,12th Street,yellow,orange,282
...,...,...,...,...
203,West Oakland,red,green,283
204,West Oakland,red,yellow,283
205,West Oakland,yellow,blue,283
206,West Oakland,yellow,green,283


# 4. Calculate weight called "combined_weight" based on average Exit counts and travel_time for the destination station

In [9]:
rollback_before_flag = True
rollback_after_flag = True

query = """

WITH q AS (
    SELECT DISTINCT a.line, a.station as from_station, b.station as to_station, t.travel_time,
    MIN(t.travel_time) OVER (PARTITION BY a.station, b.station) as min_travel_time,
    MAX(t.travel_time) OVER (PARTITION BY a.station, b.station) as max_travel_time
    FROM lines a
    JOIN lines b ON a.line = b.line AND b.sequence = (a.sequence + 1)
    JOIN travel_times t ON (a.station = t.station_1 AND b.station = t.station_2)
        OR (a.station = t.station_2 AND b.station = t.station_1)
),
be_agg AS (
    SELECT station, MIN(average_exit_count) as min_exit_count, MAX(average_exit_count) as max_exit_count
    FROM stage_bart_exits
    GROUP BY station
),
min_max_values AS (
    SELECT 
        MIN(q.travel_time) AS min_travel_time,
        MAX(q.travel_time) AS max_travel_time,
        MIN(be.average_exit_count) AS min_exit_count,
        MAX(be.average_exit_count) AS max_exit_count
    FROM q
    LEFT JOIN stage_bart_exits be ON q.to_station = be.station
)
SELECT 
    q.from_station,
    q.to_station,
    MAX(q.travel_time) AS travel_time,
    AVG(be.average_exit_count) AS average_exit_count,
    (0.6 * ((MAX(q.travel_time) - min_max_values.min_travel_time) / (min_max_values.max_travel_time - min_max_values.min_travel_time))) +
    (0.4 * ((AVG(be.average_exit_count) - min_max_values.min_exit_count) / (min_max_values.max_exit_count - min_max_values.min_exit_count))) AS combined_weight
FROM stage_bart_exits be
LEFT JOIN stations s ON be.station = s.station
JOIN q ON be.station = q.to_station
CROSS JOIN min_max_values
GROUP BY q.from_station, q.to_station, min_max_values.min_travel_time, min_max_values.max_travel_time, min_max_values.min_exit_count, min_max_values.max_exit_count
ORDER BY combined_weight desc;


"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,from_station,to_station,travel_time,average_exit_count,combined_weight
0,West Oakland,Embarcadero,420,15097,0.800000
1,West Dublin,Castro Valley,600,1046,0.615147
2,Milpitas,Warm Springs,540,1079,0.549384
3,Warm Springs,Milpitas,540,919,0.545001
4,OAK,Coliseum,480,2376,0.518242
5,San Bruno,Millbrae,420,1772,0.435031
6,Lake Merritt,West Oakland,360,2928,0.400027
7,Antioch,Pittsburg Center,420,493,0.400000
8,Concord,Pleasant Hill,360,2050,0.375979
9,Warm Springs,Fremont,360,1734,0.367324
